In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
from ta import *
from sklearn.metrics import confusion_matrix
import pylab as pl

# from tf.keras.models import Sequential  # This does not work!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau

In [2]:
tf.__version__

'1.4.0'

In [3]:
tf.keras.__version__

'2.0.8-tf'

In [4]:
pd.__version__

'0.20.3'

In [5]:
#Carregar dados
stock = pd.read_csv('dow_jones.csv')

In [6]:
stock.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1980-01-02,0.718750,0.721354,0.703125,0.703125,0.018480,3451200
1,1980-01-03,0.703125,0.723958,0.697917,0.721354,0.018959,3960000
2,1980-01-04,0.721354,0.729167,0.718750,0.729167,0.019165,1694400
3,1980-01-07,0.729167,0.731771,0.723958,0.726563,0.019096,4396800
4,1980-01-08,0.726563,0.734375,0.726563,0.731771,0.019233,3244800


In [7]:
# Utilizando a livaria 'ta'
# Add bollinger band high indicator filling NaN values
stock['bb_high_indicator'] = bollinger_hband_indicator(stock["Close"], n=20, ndev=2, fillna=True)
# Add bollinger band low indicator filling NaN values
stock['bb_low_indicator'] = bollinger_lband_indicator(stock["Close"], n=20, ndev=2, fillna=True)
# Adicionar Money Flow Index
stock['MoneyFlowIndex'] = money_flow_index(stock["High"],stock["Low"],stock["Close"],stock["Volume"],n=14, fillna=True)
# Adicionar AwesomeOscillator
stock['AwesomeOscillator'] = ao(stock["High"],stock["Low"], s=5, l=34, fillna=True)
# Adicionar Stochatic Oscillator
stock['StochaticOscillator'] = stoch(stock["High"],stock["Low"],stock["Close"], n=14, fillna=True)
# Adicionar Stochatic Oscillator Signal
stock['StochaticOscillatorSignal'] = stoch_signal(stock["High"],stock["Low"],stock["Close"], n=14, d_n=3, fillna=True)
# Adicionar True Strength Index
stock['TrueStrengthIndex'] = tsi(stock["Close"], r=25, s=13, fillna=True)
# Adicionar Ultimate Oscillator
stock['UltimateOscillator'] = uo(stock["High"],stock["Low"],stock["Close"], s=7, m=14, l=28, ws=4.0, wm=2.0, wl=1.0, fillna=True)
# Adicionar William % R
stock['William%R'] = wr(stock["High"],stock["Low"],stock["Close"], lbp=14, fillna=True)
# Adicionar RSI
stock['RSI'] = rsi(stock["Close"], n=14, fillna=True)

# Adicionar features (MM4,MM9,MM18) e uma label (1=Subiu, 0=Desceu)

stock['Change'] = stock['High'].pct_change()
stock['MM4'] = stock['Close'].rolling(window=4).mean()
stock['MM9'] = stock['Close'].rolling(window=9).mean()
stock['MM18'] = stock['Close'].rolling(window=18).mean()
stock['CrossMM'] = np.where(stock['MM4'] > stock['MM18'],1.0,0.0)
stock['CrossPrice'] = np.where(stock['Open'] > stock['MM18'],1.0,0.0)
    
# Preencher os dados em falta
stock['UP'] = np.where(stock['Change'] > 0.00, 1, 0)
media4 = stock['MM4'].mean()
media9 = stock['MM9'].mean()
media18 = stock['MM18'].mean()
stock['MM4'].fillna(media4, inplace=True)
stock['MM9'].fillna(media9, inplace=True)
stock['MM18'].fillna(media18, inplace=True)
stock['Change'].fillna(0, inplace=True) 

In [8]:
stock.UP = stock.UP.shift(1)
stock['UP'].fillna(1,inplace=True)
stock.head()

,Date,Open,High,Low,Close,Adj Close,Volume,bb_high_indicator,bb_low_indicator,MoneyFlowIndex,...,UltimateOscillator,William%R,RSI,Change,MM4,MM9,MM18,CrossMM,CrossPrice,UP
0,1980-01-02,0.718750,0.721354,0.703125,0.703125,0.018480,3451200,0.0,0.0,50.0,...,50.0,-50.0,50.000000,0.000000,36.396759,36.397601,36.399115,0.0,0.0,1.0
1,1980-01-03,0.703125,0.723958,0.697917,0.721354,0.018959,3960000,0.0,0.0,50.0,...,50.0,-50.0,100.000000,0.003610,36.396759,36.397601,36.399115,0.0,0.0,0.0
2,1980-01-04,0.721354,0.729167,0.718750,0.729167,0.019165,1694400,0.0,0.0,50.0,...,50.0,-50.0,100.000000,0.007195,36.396759,36.397601,36.399115,0.0,0.0,1.0
3,1980-01-07,0.729167,0.731771,0.723958,0.726563,0.019096,4396800,0.0,0.0,50.0,...,50.0,-50.0,88.711278,0.003571,0.720052,36.397601,36.399115,0.0,0.0,1.0
4,1980-01-08,0.726563,0.734375,0.726563,0.731771,0.019233,3244800,0.0,0.0,50.0,...,50.0,-50.0,91.044315,0.003558,0.727214,36.397601,36.399115,0.0,0.0,1.0


In [9]:
stock.keys()

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'bb_high_indicator', 'bb_low_indicator', 'MoneyFlowIndex',
       'AwesomeOscillator', 'StochaticOscillator', 'StochaticOscillatorSignal',
       'TrueStrengthIndex', 'UltimateOscillator', 'William%R', 'RSI', 'Change',
       'MM4', 'MM9', 'MM18', 'CrossMM', 'CrossPrice', 'UP'],
      dtype='object')

In [10]:
corr_matrix = stock.corr()
corr_matrix["UP"].sort_values(ascending=False)

UP                           1.000000
RSI                          0.339836
William%R                    0.317717
StochaticOscillator          0.317717
StochaticOscillatorSignal    0.303840
CrossPrice                   0.273471
UltimateOscillator           0.233278
MoneyFlowIndex               0.187367
bb_high_indicator            0.170578
CrossMM                      0.155692
Change                       0.065955
TrueStrengthIndex            0.046477
Low                          0.039495
Open                         0.039046
Close                        0.038991
High                         0.038692
Adj Close                    0.037722
AwesomeOscillator            0.037302
MM4                          0.034319
MM9                          0.030899
MM18                         0.029501
Volume                       0.012527
bb_low_indicator            -0.138687
Name: UP, dtype: float64

In [11]:
#Input dados
stock_new = stock[['Volume','High','Low','RSI','CrossMM','CrossPrice','bb_high_indicator','MoneyFlowIndex','Open','MM4','MM9','MM18','StochaticOscillator', 'William%R', 'UltimateOscillator', 'StochaticOscillatorSignal' ]]
#target
target_name = stock['UP']

In [12]:
#NumPy Arrays
#We now convert the Pandas data-frames to NumPy arrays that can be input to the neural network.
#These are the input-signals:

x_data = stock_new.values[:]
print(type(x_data))
print("Shape:", x_data.shape)

<class 'numpy.ndarray'>
Shape: (89604, 16)


In [13]:
#These are the output-signals (or target-signals):

y_data = target_name.values[:]
print(type(y_data))
print("Shape:", y_data.shape)

<class 'numpy.ndarray'>
Shape: (89604,)


In [14]:
#This is the number of observations (aka. data-points or samples) in the data-set:
num_data = len(x_data)
num_data

89604

In [15]:
#This is the fraction of the data-set that will be used for the training-set:
train_split = 0.9

In [16]:
#This is the number of observations in the training-set:
num_train = int(train_split * num_data)
num_train

80643

In [17]:
#This is the number of observations in the test-set:
num_test = num_data - num_train
num_test

8961

In [18]:
#These are the input-signals for the training- and test-sets:
x_train = x_data[0:num_train]
x_test = x_data[num_train:]
len(x_train) + len(x_test)

89604

In [19]:
#These are the output-signals for the training- and test-sets:
y_train = y_data[0:num_train]
y_test = y_data[num_train:]
len(y_train) + len(y_test)

89604

In [20]:
#This is the number of input-signals:
num_x_signals = x_data.shape[1]
num_x_signals

16

In [21]:
#This is the number of output-signals:
num_y_signals = 2

In [22]:
#Scaled Data
#The data-set contains a wide range of values:

print("Min:", np.min(x_train))
print("Max:", np.max(x_train))

Min: -100.0
Max: 1031788800.0


In [23]:
#The neural network works best on values roughly between -1 and 1, so we need to scale the data before it is being input 
#to the neural network. We can use scikit-learn for this.
#We first create a scaler-object for the input-signals

x_scaler = MinMaxScaler()

In [24]:
#We then detect the range of values from the training-data and scale the training-data.
x_train_scaled = x_scaler.fit_transform(x_train)

In [25]:
#Apart from a small rounding-error, the data has been scaled to be between 0 and 1.
print("Min:", np.min(x_train_scaled))
print("Max:", np.max(x_train_scaled))

Min: 0.0
Max: 1.0


In [26]:
#We use the same scaler-object for the input-signals in the test-set.
x_test_scaled = x_scaler.transform(x_test)

In [27]:
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1,1)
y_scaler = MinMaxScaler()
y_train_scaled = y_scaler.fit_transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

In [28]:
print(x_train_scaled.shape)
print(y_train_scaled.shape)

(80643, 16)
(80643, 1)


In [29]:
#Instead of training the Recurrent Neural Network on the complete sequences of almost 8000 observations, we will use the 
#following function to create a batch of shorter sub-sequences picked at random from the training-data.

def batch_generator(batch_size, sequence_length):
    """
    Generator function for creating random batches of training-data.
    """

    # Infinite loop.
    while True:
        # Allocate a new array for the batch of input-signals.
        x_shape = (batch_size, sequence_length, num_x_signals)
        x_batch = np.zeros(shape=x_shape, dtype=np.float16)

        # Allocate a new array for the batch of output-signals.
        y_shape = (batch_size, sequence_length, num_y_signals)
        y_batch = np.zeros(shape=y_shape, dtype=np.float16)

        # Fill the batch with random sequences of data.
        for i in range(batch_size):
            # Get a random start-index.
            # This points somewhere into the training-data.
            idx = np.random.randint(num_train - sequence_length)
            
            # Copy the sequences of data starting at this index.
            x_batch[i] = x_train_scaled[idx:idx+sequence_length]
            y_batch[i] = y_train_scaled[idx:idx+sequence_length]
        
        yield (x_batch, y_batch)

In [30]:
#batch_size = 256
batch_size = 50

In [31]:
#sequence_length = 450
sequence_length = 75
sequence_length

75

In [32]:
#We then create the batch-generator.
generator = batch_generator(batch_size=batch_size, sequence_length=sequence_length)

In [33]:
#We can then test the batch-generator to see if it works.
x_batch, y_batch = next(generator)

In [34]:
print(x_batch.shape)
print(y_batch.shape)

(50, 75, 16)
(50, 75, 2)


In [35]:
"""
Validation Set:

The neural network trains quickly so we can easily run many training epochs. But then there is a risk of overfitting the 
model to the training-set so it does not generalize well to unseen data. We will therefore monitor the model's performance 
on the test-set after each epoch and only save the model's weights if the performance is improved on the test-set.

The batch-generator randomly selects a batch of short sequences from the training-data and uses that during training. But for
the validation-data we will instead run through the entire sequence from the test-set and measure the prediction accuracy on 
that entire sequence.
"""

validation_data = (np.expand_dims(x_test_scaled, axis=0), np.expand_dims(y_test_scaled, axis=0))

In [36]:
"""
Create the Recurrent Neural Network:

We are now ready to create the Recurrent Neural Network (RNN). We will use the Keras API for this because of its simplicity. 
"""

model = Sequential()

In [37]:
"""
We can now add a Gated Recurrent Unit (GRU) to the network. This will have 512 outputs for each time-step in the sequence.

Note that because this is the first layer in the model, Keras needs to know the shape of its input, which is a batch of 
sequences of arbitrary length (indicated by None), where each observation has a number of input-signals (num_x_signals).
"""

model.add(GRU(units=512, return_sequences=True, input_shape=(None, num_x_signals,)))

In [38]:
model.add(Dense(num_y_signals, activation='sigmoid'))

In [39]:
if False:
    from tensorflow.python.keras.initializers import RandomUniform

    # Maybe use lower init-ranges.
    init = RandomUniform(minval=-0.05, maxval=0.05)

    model.add(Dense(num_y_signals,activation='linear',kernel_initializer=init))

In [40]:
warmup_steps = 10

In [41]:
def loss_mse_warmup(y_true, y_pred):
    """
    Calculate the Mean Squared Error between y_true and y_pred,
    but ignore the beginning "warmup" part of the sequences.
    
    y_true is the desired output.
    y_pred is the model's output.
    """

    # The shape of both input tensors are:
    # [batch_size, sequence_length, num_y_signals].

    # Ignore the "warmup" parts of the sequences
    # by taking slices of the tensors.
    y_true_slice = y_true[:, warmup_steps:, :]
    y_pred_slice = y_pred[:, warmup_steps:, :]

    # These sliced tensors both have this shape:
    # [batch_size, sequence_length - warmup_steps, num_y_signals]

    # Calculate the MSE loss for each value in these tensors.
    # This outputs a 3-rank tensor of the same shape.
    loss = tf.losses.mean_squared_error(labels=y_true_slice,
                                        predictions=y_pred_slice)

    # Keras may reduce this across the first axis (the batch)
    # but the semantics are unclear, so to be sure we use
    # the loss across the entire tensor, we reduce it to a
    # single scalar with the mean function.
    loss_mean = tf.reduce_mean(loss)

    return loss_mean

In [42]:
optimizer = RMSprop(lr=1e-3)

In [43]:
#We then compile the Keras model so it is ready for training.
model.compile(loss=loss_mse_warmup, optimizer=optimizer)

In [44]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, None, 512)         812544    
_________________________________________________________________
dense_1 (Dense)              (None, None, 2)           1026      
Total params: 813,570
Trainable params: 813,570
Non-trainable params: 0
_________________________________________________________________


In [45]:
path_checkpoint = '23_checkpoint.keras'
callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
                                      monitor='val_loss',
                                      verbose=1,
                                      save_weights_only=True,
                                      save_best_only=True)

In [46]:
#This is the callback for stopping the optimization when performance worsens on the validation-set.
callback_early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

In [47]:
#This is the callback for writing the TensorBoard log during training.
callback_tensorboard = TensorBoard(log_dir='./23_logs/',histogram_freq=0, write_graph=False)

In [48]:
callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                       factor=0.1,
                                       min_lr=1e-4,
                                       patience=0,
                                       verbose=1)

In [49]:
callbacks = [callback_early_stopping,
             callback_checkpoint,
             callback_tensorboard,
             callback_reduce_lr]

In [50]:
%%time
model.fit_generator(generator=generator,
                    #epochs=20,
                    epochs=15,
                    #steps_per_epoch=100,
                    steps_per_epoch=25,
                    validation_data=validation_data,
                    callbacks=callbacks)

Epoch 1/15
25/25 [==============================] - 59s - loss: 0.2202 - val_loss: 0.1873
Epoch 2/15
25/25 [==============================] - 53s - loss: 0.1761 - val_loss: 0.1711
Epoch 3/15
25/25 [==============================] - 53s - loss: 0.1661 - val_loss: 0.1572
Epoch 4/15
24/25 [===========================>..] - ETA: 2s - loss: 0.1592Epoch 00003: val_loss did not improve

Epoch 00003: reducing learning rate to 0.00010000000474974513.
25/25 [==============================] - 55s - loss: 0.1592 - val_loss: 0.1583
Epoch 5/15
25/25 [==============================] - 54s - loss: 0.1511 - val_loss: 0.1476
Epoch 6/15
25/25 [==============================] - 56s - loss: 0.1475 - val_loss: 0.1466
Epoch 7/15
25/25 [==============================] - 57s - loss: 0.1464 - val_loss: 0.1462
Epoch 8/15
25/25 [==============================] - 58s - loss: 0.1460 - val_loss: 0.1454
Epoch 9/15
25/25 [==============================] - 58s - loss: 0.1458 - val_loss: 0.1438
Epoch 10/15
25/25 [======

In [51]:
try:
    model.load_weights(path_checkpoint)
except Exception as error:
    print("Error trying to load checkpoint.")
    print(error)

In [52]:
"""
Performance on Test-Set:

We can now evaluate the model's performance on the test-set. This function expects a batch of data, but we will just use one 
long time-series for the test-set, so we just expand the array-dimensionality to create a batch with that one sequence.
"""
result = model.evaluate(x=np.expand_dims(x_test_scaled, axis=0),
                        y=np.expand_dims(y_test_scaled, axis=0))
print(result)

1/1 [==============================] - 5s
0.138835772872


In [53]:
print("loss (test-set):", result)
# If you have several metrics you can use this instead.
if False:
    for res, metric in zip(result, model.metrics_names):
        print("{0}: {1:.3e}".format(metric, res))

loss (test-set): 0.138835772872


In [54]:
def plot_comparison(start_idx, length=100, train=True):
    """
    Plot the predicted and true output-signals.
    
    :param start_idx: Start-index for the time-series.
    :param length: Sequence-length to process and plot.
    :param train: Boolean whether to use training- or test-set.
    """
    
    if train:
        # Use training-data.
        x = x_train_scaled
        y_true = y_train
    else:
        # Use test-data.
        x = x_test_scaled
        y_true = y_test
    
    # End-index for the sequences.
    end_idx = start_idx + length
    
    # Select the sequences from the given start-index and
    # of the given length.
    x = x[start_idx:end_idx]
    y_true = y_true[start_idx:end_idx]
    
    # Input-signals for the model.
    x = np.expand_dims(x, axis=0)

    # Use the model to predict the output-signals.
    y_pred = model.predict(x)
    # The output of the model is between 0 and 1.
    # Do an inverse map to get it back to the scale
    # of the original data-set.
    y_pred_rescaled = y_scaler.inverse_transform(y_pred[0])  
    print('Previsoes:', y_pred_rescaled)
    print('Real:', y_true)  
    
    """
    # For each output-signal.
    for signal in range(len(target_name)):
        # Get the output-signal predicted by the model.
        signal_pred = y_pred_rescaled[:, signal]
        
        # Get the true output-signal from the data-set.
        signal_true = y_true[:, signal]

        # Make the plotting-canvas bigger.
        plt.figure(figsize=(15,5))
        
        # Plot and compare the two signals.
        plt.plot(signal_true, label='true')
        plt.plot(signal_pred, label='pred')
        
        # Plot grey box for warmup-period.
        p = plt.axvspan(0, warmup_steps, facecolor='black', alpha=0.15)
        
        # Plot labels etc.
        plt.ylabel(target_name[signal])
        plt.legend()
        plt.show()
     """   

In [55]:
plot_comparison(start_idx=8810, length=10, train=False)

Previsoes: [[ 0.52386218  0.51255918]
 [ 0.98990029  0.98945528]
 [ 0.98816073  0.98848104]
 [ 0.68588912  0.69752252]
 [ 0.39837825  0.4113231 ]
 [ 0.61949348  0.62437916]
 [ 0.15522966  0.1557842 ]
 [ 0.10198675  0.10242577]
 [ 0.24104708  0.24018288]
 [ 0.9428677   0.94364852]]
Real: [[ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]]


In [56]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#from stockstats import StockDataFrame as Sdf
from ta import *
%matplotlib inline

nome_ficheiro = 'lista_dax.txt'
#nome_ficheiro = 'lista_dowj.csv'

try:
        fhand = open(nome_ficheiro)

except:
        print ('Nao e possivel abrir o ficheiro: ', fhand)


lista_quotes = []

#Cria lista com os ticket do SP500
for f in fhand:
    linha = f.strip().split(',')
    ticker = linha[0]
    lista_quotes.append(ticker)


for quote in lista_quotes:
    try:
        stock = pd.read_csv('dados_actuais_'+quote+'_280419.csv')
        #stock = pd.read_csv('dados_actuais_dowj_'+quote+'_260419.csv')
        output = stock
    except:
        continue    
    
    # Utilizando a livaria 'ta'
    # Add bollinger band high indicator filling NaN values
    output['bb_high_indicator'] = bollinger_hband_indicator(output["Close"], n=20, ndev=2, fillna=True)
    # Add bollinger band low indicator filling NaN values
    output['bb_low_indicator'] = bollinger_lband_indicator(output["Close"], n=20, ndev=2, fillna=True)
    # Adicionar Money Flow Index
    output['MoneyFlowIndex'] = money_flow_index(output["High"],output["Low"],output["Close"],output["Volume"],n=14, fillna=True)
    # Adicionar AwesomeOscillator
    output['AwesomeOscillator'] = ao(output["High"],output["Low"], s=5, l=34, fillna=True)
    # Adicionar Stochatic Oscillator
    output['StochaticOscillator'] = stoch(output["High"],output["Low"],output["Close"], n=14, fillna=True)
    # Adicionar Stochatic Oscillator Signal
    output['StochaticOscillatorSignal'] = stoch_signal(output["High"],output["Low"],output["Close"], n=14, d_n=3, fillna=True)
    # Adicionar True Strength Index
    output['TrueStrengthIndex'] = tsi(output["Close"], r=25, s=13, fillna=True)
    # Adicionar Ultimate Oscillator
    output['UltimateOscillator'] = uo(output["High"],output["Low"],output["Close"], s=7, m=14, l=28, ws=4.0, wm=2.0, wl=1.0, fillna=True)
    # Adicionar William % R
    output['William%R'] = wr(output["High"],output["Low"],output["Close"], lbp=14, fillna=True)
    # Adicionar RSI
    output['RSI'] = rsi(output["Close"], n=14, fillna=True)
    
    # Adicionar features (MM4,MM9,MM18) e uma label (1=Subiu, 0=Desceu)

    output['Change'] = output['Close'].pct_change()
    output['UP'] = np.where(output['Change'] >= 0.0,1,0)
    output['MM4'] = output['Close'].rolling(window=4).mean()
    output['MM9'] = output['Close'].rolling(window=9).mean()
    output['MM18'] = output['Close'].rolling(window=18).mean()
    output['CrossMM'] = np.where(output['MM4'] > output['MM18'],1.0,0.0)
    output['CrossPrice'] = np.where(output['Open'] > output['MM18'],1.0,0.0)
    
    # Preencher os dados em falta

    media4 = output['MM4'].mean()
    media9 = output['MM9'].mean()
    media18 = output['MM18'].mean()
    output['MM4'].fillna(media4, inplace=True)
    output['MM9'].fillna(media9, inplace=True)
    output['MM18'].fillna(media18, inplace=True)
    output['Change'].fillna(0, inplace=True)    
    
    
    # Criar tabela com 'features', e com respectiva label (UP)
    y = output['UP']
    X = output[['Volume','High','Low','RSI','CrossMM','CrossPrice','bb_high_indicator','MoneyFlowIndex','Open','MM4','MM9','MM18','StochaticOscillator', 'William%R', 'UltimateOscillator', 'StochaticOscillatorSignal' ]]

    #Split 
    
    tamanho_t = len(X)
    terco = int(tamanho_t/3)
    tr = tamanho_t - terco
    
    
    x_train = X.head(tr)
    x_test  = X.tail(terco)
    y_train = y.head(tr)
    y_test = y.tail(terco)
    
    
    x_scaler = MinMaxScaler()
    x_train_scaled = x_scaler.fit_transform(x_train)
    x_test_scaled = x_scaler.transform(x_test)
    
          
    
    """
    #Scaling

    sc = StandardScaler()
    X_train_sca = sc.fit_transform(X_train)
    X_test_sca = sc.transform(X_test)
    """
    
    
    def comparison(start_idx, length=100, train=True):
        """
        Plot the predicted and true output-signals.

        :param start_idx: Start-index for the time-series.
        :param length: Sequence-length to process and plot.
        :param train: Boolean whether to use training- or test-set.
        """

        if train:
            # Use training-data.
            x = x_train_scaled
            y_true = y_train
        else:
            # Use test-data.
            x = x_test_scaled
            y_true = y_test

        # End-index for the sequences.
        end_idx = start_idx + length

        # Select the sequences from the given start-index and
        # of the given length.
        x = x[start_idx:end_idx]
        y_true = y_true[start_idx:end_idx]

        # Input-signals for the model.
        x = np.expand_dims(x, axis=0)

        # Use the model to predict the output-signals.
        y_pred = model.predict(x)
        # The output of the model is between 0 and 1.
        # Do an inverse map to get it back to the scale
        # of the original data-set.
        y_pred_rescaled = y_scaler.inverse_transform(y_pred[0])  
        return y_pred_rescaled      
    
    
    
    x_pred_rescaled = comparison(start_idx=1, length=500, train=False)
    
    
    #y_pred = model.predict(x)    
    #y_pred = model.predict(x_test_sca).astype(np.double).round(decimals=1)[-5:]
    print('Valor Binário (0)', quote, '-->', x_pred_rescaled.astype(np.double).round(decimals=1)[-5:])
    

Valor Binário (0) ADS.DE --> [[ 0.7  0.7]
 [ 1.   1. ]
 [ 1.   1. ]
 [ 0.5  0.5]
 [ 0.   0. ]]
Valor Binário (0) ALV.DE --> [[ 1.   1. ]
 [ 1.   1. ]
 [ 0.9  0.9]
 [ 0.2  0.2]
 [ 0.2  0.2]]
Valor Binário (0) BAS.DE --> [[ 1.   1. ]
 [ 0.9  0.9]
 [ 1.   1. ]
 [ 0.8  0.8]
 [ 0.   0. ]]
Valor Binário (0) BAYN.DE --> [[ 0.6  0.6]
 [ 0.1  0.1]
 [ 0.6  0.6]
 [ 0.   0. ]
 [ 0.   0. ]]
Valor Binário (0) BEI.DE --> [[ 0.   0. ]
 [ 0.9  0.9]
 [ 0.9  0.9]
 [ 0.7  0.7]
 [ 0.4  0.4]]
Valor Binário (0) BMW.DE --> [[ 1.   1. ]
 [ 0.9  0.9]
 [ 0.2  0.2]
 [ 0.1  0.1]
 [ 0.   0. ]]
Valor Binário (0) CBK.DE --> [[ 1.   1. ]
 [ 0.7  0.6]
 [ 0.1  0.1]
 [ 0.1  0.1]
 [ 0.2  0.2]]
Valor Binário (0) CON.DE --> [[ 1.   1. ]
 [ 0.9  0.9]
 [ 0.3  0.3]
 [ 0.   0. ]
 [ 0.   0. ]]
Valor Binário (0) DAI.DE --> [[ 1.   1. ]
 [ 1.   1. ]
 [ 0.8  0.8]
 [ 0.   0. ]
 [ 0.   0. ]]
Valor Binário (0) DB1.DE --> [[ 0.9  0.9]
 [ 0.9  0.9]
 [ 0.9  0.9]
 [ 0.7  0.7]
 [ 0.2  0.1]]
Valor Binário (0) DBK.DE --> [[ 1.   1. ]
 [ 0.5 